### TinyStories: генерация коротких текстов

В этом задании вы реализуете transformer decoder и обучите его на датасете TinyStories, а также продемонстрируете его работу, сгенерировав несколько примеров текстов по заданному началу.

Большая часть блоков уже описана в слайдах и в ноутбуках [notebooks/transformer_attention.ipynb](../notebooks/transformer_attention.ipynb) и [notebooks/transformer_model.ipynb](../notebooks/transformer_model.ipynb), вам осталось только собрать их в модуль `TransformerDecoder`.

In [1]:
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch import nn, Tensor
from torch.utils.data import DataLoader
from transformers import GPT2TokenizerFast

Загрузим датасет:

In [2]:
ds = load_dataset("roneneldan/TinyStories")
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

Посмотрим на несколько примеров:

In [3]:
ds["train"]["text"][:4]

['One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.',
 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that we

Будем использовать готовый токенизатор, например `GPT2TokenizerFast`:

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [5]:
inputs = tokenizer.encode("Once upon a time", return_tensors="pt", add_special_tokens=True)
inputs

tensor([[7454, 2402,  257,  640]])

У этого токенизатора нет выделенного токена для начала последовательности, есть только один специальный токен `<|endoftext|>`. Поэтому, если вы захотите добавить его к началу предложения, придётся делать это вручную:

In [6]:
tokenizer.encode("<|endoftext|>Once upon a time", return_tensors="pt", add_special_tokens=True)

tensor([[50256,  7454,  2402,   257,   640]])

Вместо этого можно добавлять его индекс `50256` в начало всех тензоров уже после токенизации.

Токенизатор можно использовать прямо в загрузчике данных для упаковки токенизированных последовательностей в батчи.

Помимо индексов токенов (`input_ids`) токенизатор вернёт нам `attention_mask` — маску, в которой нулями помечены `pad` токены, использующиеся для выравнивания предложений по длине.

In [7]:
loader = DataLoader(
    dataset=ds["train"]["text"],
    collate_fn=lambda batch: tokenizer(batch, return_tensors="pt", padding=True),
    batch_size=8,
)
batch = next(iter(loader))
batch

{'input_ids': tensor([[ 3198,  1110,    11,  ..., 50256, 50256, 50256],
        [ 7454,  2402,   257,  ..., 50256, 50256, 50256],
        [ 3198,  1110,    11,  ...,   922,  2460,    13],
        ...,
        [ 7454,  2402,   257,  ..., 50256, 50256, 50256],
        [ 7454,  2402,   257,  ..., 50256, 50256, 50256],
        [ 7454,  2402,   257,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

#### Задание 1. Обучение Transformer Decoder (4 балла)

1. Реализуйте модуль `TransformerDecoder`. Не забудьте сформировать правильную маску внимания! Для этого можете воспользоваться функцией `create_causal_mask` ниже.
2. Создайте с его помощью модель небольшого размера (до 30 млн параметров) и обучите её на `train` части датасета TinyStories. Обучение на всём датасете может занять слишком много времени, так что сделайте столько итераций градиентного спуска, сколько уместится в 10-15 минут обучения. Кроме того, вы можете сделать подвыборку датасета с более короткими текстами, это ускорит обучение модели. Относительно связный текст обычно начинает получаться при снижении cross entropy до 1.5-2.0.
3. После окончания обучения сгенерируйте 10 историй с помощью вашей модели.


In [ ]:
def create_causal_mask(token_ids: Tensor, pad_token_id: int = 0) -> Tensor:
    B, T = token_ids.shape
    # маска для <pad> токенов
    pad_mask = (token_ids != pad_token_id).unsqueeze(1)  # B x 1 x T
    # маска нижнетреугольной матрицы
    causal_mask = torch.tril(
        torch.ones((T, T), device=token_ids.device)
    ).bool()  # T x T
    return pad_mask & causal_mask  # B x T x T


class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size: int, hidden_size: int, n_layers: int):
        ...

    def forward(self, input_ids: Tensor, attention_mask: Tensor) -> Tensor:
        ...

Для генерации примеров можно использовать простейшую стратегию с семплированием следующего токена:

In [8]:
def generate(
    model: TransformerDecoder, input_ids: Tensor, max_new_tokens: int = 200
) -> Tensor:
    for t in range(max_new_tokens):
        mask = create_causal_mask(input_ids)
        logits = model.forward(input_ids, mask)[:, -1]
        # new_token = logits.argmax(dim=-1, keepdim=True)
        new_token = torch.multinomial(logits.softmax(-1), num_samples=1)
        input_ids = torch.cat([input_ids, new_token], dim=1)

    return input_ids

In [ ]:
prompt = "Once upon a time"
inputs = tokenizer(prompt, return_tensors="pt").to(device="mps")
generate_ids = generate(model, inputs.input_ids, max_new_tokens=200)
response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(response)